<a href="https://colab.research.google.com/github/dvillasxUdg/ti_p4/blob/main/TI_1_Practica4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 4

**Nombre:** Damian Alfonso Villaseñor Cisneros  
**e-mail:** damian.villasenor3846@alumnos.udg.mx

## MODULES

In [ ]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

## CLASSES

In [ ]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [ ]:
def bm_2d(n_steps=1000, speed=6, s_pos=[0,0]):
  """
  Arguments:
    n_steps (int): number of steps to walk
    speed (float): speed of the walker
    s_pos (list): init position for [x, y].
  Returns:
    BM_2d_df(DataFrame): DataFrame with a BM
  """

  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

In [ ]:
def crw_2d(n_steps=1000, speed=6, s_pos=[0, 0], crw_exponent=0.7):
    """
    Arguments:
        n_steps (int): number of steps to walk
        speed (float): speed of the walker
        s_pos (list): init position for [x, y].
        crw_exponent (float): wrapcauchy distribution exponent

    Returns:
        crw_2d_df(DataFrame): DataFrame with a CRW
    """
    velocity = Vec2d(speed, 0)

    crw_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

    crw_2d_df = pd.concat([crw_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps - 1):
        turn_angle = wrapcauchy.rvs(crw_exponent)
        velocity = velocity.rotated(turn_angle)

        s_pos = [s_pos[0] + velocity.x, s_pos[1] + velocity.y]

        temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
        crw_2d_df = pd.concat([crw_2d_df, temp_df], ignore_index=True)

    return crw_2d_df

In [ ]:
def lw_2d(n_steps=1000, speed=6, s_pos=[0, 0], crw_exponent=0.9, alpha=1.9, beta=0.0, loc=3.0):
    velocity = Vec2d(speed, 0)
    levy_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
    levy_2d_df = pd.concat([levy_2d_df, temp_df], ignore_index=True)

    total_steps = 0

    while total_steps < n_steps-1:
        turn_angle = wrapcauchy.rvs(crw_exponent)
        step = int(levy_stable.rvs(alpha=alpha, beta=beta, loc=loc))

        if total_steps + step > n_steps:
            step = n_steps - total_steps

        velocity = velocity.rotated(turn_angle)

        for _ in range(step):
            s_pos = [s_pos[0] + velocity.x, s_pos[1] + velocity.y]
            temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
            levy_2d_df = pd.concat([levy_2d_df, temp_df], ignore_index=True)

        total_steps += step

    return levy_2d_df